In [399]:
import trello
import re
import arrow

from dateutil import tz
from collections import defaultdict

## Functions

In [462]:
def parse_qty(q):
    re_frac = re.compile(r"(\d+.?\d*)\/(\d+.?\d*)")
    m = re_frac.match(q)
    if m:
        return float(m.group(1)) / float(m.group(2))
    else:
        try:
            return float(q)
        except ValueError as err:
            return 0
        
def empty_ingredients():
    return defaultdict(lambda: defaultdict(float))

def get_ingredients(card):
    re_ing = re.compile(r"\- ([\*\w\s\(\)]+)\s?\[?([\d\/]*\s*\w*)\]?")
    d = card.desc.split('\n')
    try:
        begin = d.index('# Ingredients')
    except ValueError as err:
        return []
    if '# Instructions' in d:
        end = d.index('# Instructions')
        raw_ingredients = d[begin+1:end]
    else:
        raw_ingredients = d[begin+1:]
    matches = [re_ing.match(i) for i in raw_ingredients]
    tuples = [m.group(1,2) for m in matches if m is not None]
    
    # parse quantities
    ingredients = empty_ingredients()
    for ing, qty in tuples:
        # skip ingredients starting with *
        if ing[0] == '*':
            continue
        # case where no quantity is given
        ing = ing.strip()
        if not qty:
            ingredients[ing]['count'] += 1
        else:
            qty = qty.strip()
            qty_splits = qty.split()
            # simple case
            if len(qty_splits) == 1:
                ingredients[ing]['count'] += parse_qty(qty)
            # more complex case
            else:
                ingredients[ing][" ".join(qty_splits[1:])] += parse_qty(qty_splits[0])
    return ingredients

def merge_ingredients(d1, d2):
    d = d1.copy()
    # add ingredients from d2 to d1
    for ing, dict_qty in d2.items():
        for qty_unit, qty_val in dict_qty.items():
            d[ing][qty_unit] += qty_val
    return d

def day2ingredients(card, recipes):
    ingredients = empty_ingredients()
    meals = card.desc.split('\n')
    for meal in meals:
        recipe_name = meal[2:]
        if recipe_name in recipes:
            recipe_card = recipes[meal[2:]]
            ingredients = merge_ingredients(ingredients,get_ingredients(recipe_card))
    return ingredients

def week2ingredients(l, recipes):
    ingredients = empty_ingredients()
    for day in l.list_cards():
        new_ingredients = day2ingredients(day, recipes)
        ingredients = merge_ingredients(ingredients, new_ingredients)
    return(ingredients)

def ingredients2trello(ingredients, groceries):
    str_time = arrow.now(tz=tz.gettz('US/Central')).format("MM/DD/YYYY @ HH:mm")
    card_name = 'List created %s' % str_time
    card_desc = []
    for ing, ing_qty in ingredients.items():
        line = "- %s: %s" % (ing, ",".join(["%.2f %s" % (qty, unit) for unit, qty in ing_qty.items()]))
        card_desc.append(line)
    c = groceries.add_card(card_name)
    c.add_checklist('Groceries', card_desc)
    c.set_pos(0)
    
def make_list_from_week(week, recipes, groceries):
    week_ingredients = week2ingredients(week, recipes)
    ingredients2trello(week_ingredients, groceries)

In [463]:
# Connect to Trellog and load board/lists/cards
tc = trello.TrelloClient(api_key='',
                         token='')
food_board = tc.get_board('')
# create dictionary to access a list using its name
list_dict = {l.name.decode(): l for l in food_board.all_lists()}
# create a dictionary to access a recipe using its name
recipes = {c.name.decode(): c for c in list_dict['Recipes'].list_cards()}
groceries = list_dict['Groceries List']

In [465]:
week = list_dict['Week']
make_list_from_week(week, recipes, groceries)